# Assessing GDPR-Compliance in Web Applications: A Machine Learning Approach

We will assess the GDPR-compliance of web applications based on their privacy policies. We use a classification model, trained on a corpus of 18,397 natural sentences, to classify the privacy policies on whether five General Data Protection Regulation (GDPR) privacy policy core requirements are communicated in the policy.

__Relevance:__ The GDPR applies to any personal data processing of EU citizens. We aim to assess the state of GDPR-compliance in application software based on their privacy policies.

__Focus:__ web applications; as the web application paradigm is widely used due to the omnipresence of web browsers across PCs and mobile devices. In particular, we focus on organisations that provide cloud-based solutions: Cloud Computing, Cloud Data Services, Cloud Infrastructure, Cloud, Management, and Cloud Storage.


__Goal:__ to scrutinize the privacy policies of web applications using ML, to assess whether core privacy policy requirements are communicated.

#### __RQ:__ What is the state of GDPR-compliance disclosure in web applications?

---

### Step 1: collect list of companies active in the Web Apps industry

To do so we utilize the Crunchbase database that allows us to identify companies that provide webbased services, filtered on location (which in our case will be the European Union). We used 

We've imported 2792 companies using the following criteria:
- Industry: Web Services -> Cloud Computing, Cloud Data Services, Cloud Infrastructure, Cloud, Management, and Cloud Storage
- Location: USA, India, EU

---

In [2]:
import os
from newspaper import Article
from bs4 import BeautifulSoup
from six.moves.urllib.parse import urlparse
import urllib
import sys
import time
import nltk
import glob
import pandas as pd
import requests
import spacy
import random
# from googlesearch import search
from langdetect import detect
import re
import pickle
import math
import numpy as np
import collections
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from tabulate import tabulate
from IPython.display import display, HTML

### Step 2: read data

In [5]:
path = r'C:\Users\aaberkan\OneDrive - UGent\Scripts\GDPR-Compliance in Web Applications\data\Crunchbase\Productivity tools'
filenames = glob.glob(path + "/*.csv")

In [6]:
# len = 30
len(filenames)

1

In [7]:
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

In [8]:
crunch_data = pd.concat(dfs, ignore_index=True)

In [9]:
crunch_data

,Organization Name,Organization Name URL,Founded Date,Founded Date Precision,Number of Employees,Full Description,Website,Industries,Headquarters Location,Description,...,SEMrush - Monthly Visits Growth,SEMrush - Visit Duration Growth,SEMrush - Visit Duration,SEMrush - Page Views / Visit,SEMrush - Page Views / Visit Growth,SEMrush - Bounce Rate,SEMrush - Bounce Rate Growth,SEMrush - Global Traffic Rank,SEMrush - Monthly Rank Change (#),SEMrush - Monthly Rank Growth
0,Burgon & Ball,https://www.crunchbase.com/organization/burgon...,1730-01-01,year,11-50,"Founded in Sheffield in 1730, Burgon & Ball is...",https://www.burgonandball.com/,"Consumer Goods, Home and Garden, Manufacturing...","Sheffield, Sheffield, United Kingdom",Burgon & Ball manufacturer of Garden Tools to ...,...,167.15%,-25.25%,74,2.81,19.74%,48.5%,-4.68%,"1,045,844","-765,972",-42.28%
1,Witte Tools,https://www.crunchbase.com/organization/witte-...,1785-01-01,year,10001+,NaN,https://www.wittetools.com,"Manufacturing, Productivity Tools","Hagen, Nordrhein-Westfalen, Germany",Witte Tools is a tool manufacturer for automob...,...,23.02%,565.96%,626,6.00,132.59%,0%,-44.08%,"4,823,027","-216,235",-4.29%
2,WÜSTHOF,https://www.crunchbase.com/organization/wüstho...,1814-01-01,year,101-250,"WÜSTHOF products include chef's knives, asian-...",https://www.wuesthof.com/en-in/,"Consumer Goods, Manufacturing, Product Design,...","Solingen, Nordrhein-Westfalen, Germany",WÜSTHOF manufactures cutlery with precision an...,...,NaN,NaN,0,1.00,NaN,100%,NaN,"2,569,355",NaN,NaN
3,C.S. Osborne & Co.,https://www.crunchbase.com/organization/c-s-os...,1826-01-01,year,101-250,C.S. Osborne & Co. manufactures and distribute...,https://csosborne.com,"Industrial, Manufacturing, Productivity Tools,...","Harrison, New Jersey, United States",CS Osborne & Co. is a tool manufacturer.,...,198.25%,992.42%,378,1.42,41.74%,58.24%,-41.76%,"3,856,985","-2,662,737",-40.84%
4,Browns Agricultural,https://www.crunchbase.com/organization/browns...,1830-01-01,year,NaN,NaN,https://brownsagricultural.co.uk/,"Agriculture, AgTech, Farming, Machinery Manufa...","Leighton Buzzard, Bedfordshire, United Kingdom",Browns Agricultural is a machinery manufacturi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,JotForm,https://www.crunchbase.com/organization/jotform,2006-01-01,year,251-500,JotForm is an online form building tool that h...,https://www.jotform.com,"Information Technology, Productivity Tools, So...","San Francisco, California, United States",JotForm is an online form-building tool that h...,...,-2.05%,4.38%,572,1.97,-4.04%,59.53%,4.48%,"3,441",-81,-2.3%
996,Viewpath,https://www.crunchbase.com/organization/viewpath,2006-01-01,year,1-10,Viewpath is a computer software company that s...,http://www.viewpath.com,"Collaboration, Enterprise Software, Product Ma...","Seattle, Washington, United States",Viewpath is a computer software company that s...,...,NaN,NaN,0,1.00,NaN,100%,NaN,"9,546,283",NaN,NaN
997,Wide Narrow,https://www.crunchbase.com/organization/wide-n...,2006-01-01,day,11-50,The Wide Narrow software and information ecosy...,https://www.widenarrow.com/,"Analytics, Artificial Intelligence, Business I...","Stockholm, Stockholms Lan, Sweden",The Wide Narrow software and information ecosy...,...,-84.94%,-75%,83,1.60,32.42%,79.94%,1.15%,"7,805,544","4,159,456",114.08%
998,Express Mobile,https://www.crunchbase.com/organization/expres...,2006-01-01,year,1-10,Express Mobile is a technology company that sp...,http://www.xpressmo.com/,"Enterprise Software, Information Technology, M...","Larkspur, California, United States",Express Mobile is a technology company that sp...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# remove duplicates
crunch_data.drop_duplicates(inplace=True)

In [11]:
crunch_data

,Organization Name,Organization Name URL,Founded Date,Founded Date Precision,Number of Employees,Full Description,Website,Industries,Headquarters Location,Description,...,SEMrush - Monthly Visits Growth,SEMrush - Visit Duration Growth,SEMrush - Visit Duration,SEMrush - Page Views / Visit,SEMrush - Page Views / Visit Growth,SEMrush - Bounce Rate,SEMrush - Bounce Rate Growth,SEMrush - Global Traffic Rank,SEMrush - Monthly Rank Change (#),SEMrush - Monthly Rank Growth
0,Burgon & Ball,https://www.crunchbase.com/organization/burgon...,1730-01-01,year,11-50,"Founded in Sheffield in 1730, Burgon & Ball is...",https://www.burgonandball.com/,"Consumer Goods, Home and Garden, Manufacturing...","Sheffield, Sheffield, United Kingdom",Burgon & Ball manufacturer of Garden Tools to ...,...,167.15%,-25.25%,74,2.81,19.74%,48.5%,-4.68%,"1,045,844","-765,972",-42.28%
1,Witte Tools,https://www.crunchbase.com/organization/witte-...,1785-01-01,year,10001+,NaN,https://www.wittetools.com,"Manufacturing, Productivity Tools","Hagen, Nordrhein-Westfalen, Germany",Witte Tools is a tool manufacturer for automob...,...,23.02%,565.96%,626,6.00,132.59%,0%,-44.08%,"4,823,027","-216,235",-4.29%
2,WÜSTHOF,https://www.crunchbase.com/organization/wüstho...,1814-01-01,year,101-250,"WÜSTHOF products include chef's knives, asian-...",https://www.wuesthof.com/en-in/,"Consumer Goods, Manufacturing, Product Design,...","Solingen, Nordrhein-Westfalen, Germany",WÜSTHOF manufactures cutlery with precision an...,...,NaN,NaN,0,1.00,NaN,100%,NaN,"2,569,355",NaN,NaN
3,C.S. Osborne & Co.,https://www.crunchbase.com/organization/c-s-os...,1826-01-01,year,101-250,C.S. Osborne & Co. manufactures and distribute...,https://csosborne.com,"Industrial, Manufacturing, Productivity Tools,...","Harrison, New Jersey, United States",CS Osborne & Co. is a tool manufacturer.,...,198.25%,992.42%,378,1.42,41.74%,58.24%,-41.76%,"3,856,985","-2,662,737",-40.84%
4,Browns Agricultural,https://www.crunchbase.com/organization/browns...,1830-01-01,year,NaN,NaN,https://brownsagricultural.co.uk/,"Agriculture, AgTech, Farming, Machinery Manufa...","Leighton Buzzard, Bedfordshire, United Kingdom",Browns Agricultural is a machinery manufacturi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,JotForm,https://www.crunchbase.com/organization/jotform,2006-01-01,year,251-500,JotForm is an online form building tool that h...,https://www.jotform.com,"Information Technology, Productivity Tools, So...","San Francisco, California, United States",JotForm is an online form-building tool that h...,...,-2.05%,4.38%,572,1.97,-4.04%,59.53%,4.48%,"3,441",-81,-2.3%
996,Viewpath,https://www.crunchbase.com/organization/viewpath,2006-01-01,year,1-10,Viewpath is a computer software company that s...,http://www.viewpath.com,"Collaboration, Enterprise Software, Product Ma...","Seattle, Washington, United States",Viewpath is a computer software company that s...,...,NaN,NaN,0,1.00,NaN,100%,NaN,"9,546,283",NaN,NaN
997,Wide Narrow,https://www.crunchbase.com/organization/wide-n...,2006-01-01,day,11-50,The Wide Narrow software and information ecosy...,https://www.widenarrow.com/,"Analytics, Artificial Intelligence, Business I...","Stockholm, Stockholms Lan, Sweden",The Wide Narrow software and information ecosy...,...,-84.94%,-75%,83,1.60,32.42%,79.94%,1.15%,"7,805,544","4,159,456",114.08%
998,Express Mobile,https://www.crunchbase.com/organization/expres...,2006-01-01,year,1-10,Express Mobile is a technology company that sp...,http://www.xpressmo.com/,"Enterprise Software, Information Technology, M...","Larkspur, California, United States",Express Mobile is a technology company that sp...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
crunch_data.to_csv("crunch_collaboration.csv", sep='\t', header=True, index=False)

#### Clean websites list

In [13]:
websites_list = crunch_data["Website"].tolist()

In [14]:
len(websites_list)

1000

In [15]:
# remove / from the end of the string that contains the website
# websites_list = [website.rstrip(website[-1]) if (website[-1] == "/") else website for website in websites_list]
websites_list = [website.rstrip(website[-1]) if (isinstance(website, str) and website[-1] == "/") else website for website in websites_list]
# een keer extra voor het geval er een url was met // op het eind
websites_list = [website.rstrip(website[-1]) if (isinstance(website, str) and website[-1] == "/") else website for website in websites_list]

In [16]:
len(websites_list)

1000

---

### Step 3: scrape privacy policies

In [17]:
def get_privacy_policy_url(query):
    keyword_in_title = 0
    attempts = 0
    url = ""
    print("Query: " + query)
    
    try:
        query_results_list = return_google_results(query, 3, 5)
        print("Considering " + str(len(query_results_list)) + " URL(s) ...")
        for i, url in enumerate(query_results_list):
            term_in_url = 0
            attempts = attempts + 1
            print("Assessing privacy policy URL: " + url)
            
            if (re.findall('privacy', url) or re.findall('policy', url) or re.findall('gdpr', url) 
                or re.findall('terms', url) or re.findall('legal', url)): 
                print("Found relevant terms in URL! Succesful break!")
                break

#                     pass
            if keyword_in_title == 1 or attempts == 3 or i==(len(query_results_list)-1): 
                keyword_in_title = 0
                attempts = 0
                print("No results. Breaking ..")
                url = ""
#                 print(sentences)
                break   
    except Exception as e:
            print(str(e))
            pass
    return url

In [18]:
def return_google_results(keywords, num_results, attempts):
    user_agent_list = [
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]

    html_keywords = urllib.parse.quote_plus(keywords)
    sleep_init = 10
    
    url = "https://www.google.com/search?q=" + html_keywords + "&num=" + str(num_results)
    print("** Search query in URL: " + url)

    headers = {'User-Agent': random.choice(user_agent_list)}
    
    html = requests.get(url, headers=headers)

    if html.status_code == 429:
        if(attempts == 0):
            sys.exit("Too many request 429, attempted "+ str(5)+ " times, break ...")
        else:
            if 'Retry_After' in html.headers:
                print("Helaas, geen retry-after info")
            else:
                time.sleep(sleep_init)
                print("Too many requests (attempt "+ str(5 - attempts)+ "), we will attempt again in " + str(sleep_init) + " seconds")
                return_google_results(keywords, num_results, (attempts - 1))
    else: 
        pass
        
    soup = BeautifulSoup(html.text, 'html.parser')

    allData = soup.find_all("div",{"class":"g"})

    link_list = []
    print("len alldata: " + str(len(allData)))
    
    for i in range(0,len(allData)):
        link = allData[i].find('a').get('href')
        
        if(link is not None):
            if(link.find('https') != -1 and link.find('http') == 0 and link.find('aclk') == -1):
                print(link)
                link_list.append(link)
    print(link_list)
    return link_list

#### Collect privacy policy URLs

In [19]:
privacy_policies_url_list = []

In [20]:
# loop through each company URL and attempt to find the URL of the privacy policy
count_urls = 0
for i, url_company in enumerate(websites_list):    
    print(i)

#     print(len(privacy_policies_url_list))
    if(isinstance("url_company", str) is False or (url_company == url_company) is False):
        privacy_policies_url_list.append("")
    else:
        query = "site:\"" + url_company + " \"privacy policy"
        privacy_policies_url_list.append(get_privacy_policy_url(query))
        if(len(privacy_policies_url_list[-1]) > 0):
            count_urls = count_urls + 1
    print("URL count: " + str(count_urls))
    print()
    time.sleep(50)

0
Query: site:"https://www.burgonandball.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.burgonandball.com+%22privacy+policy&num=3
len alldata: 2
https://www.burgonandball.com/pages/privacy-policy
https://www.burgonandball.com/pages/gdpr-compliance
['https://www.burgonandball.com/pages/privacy-policy', 'https://www.burgonandball.com/pages/gdpr-compliance']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.burgonandball.com/pages/privacy-policy
Found relevant terms in URL! Succesful break!
URL count: 1

1
Query: site:"https://www.wittetools.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.wittetools.com+%22privacy+policy&num=3
len alldata: 2
https://www.wittetools.com/en/general-terms-conditions-of-delivery-and-payment/privacy-policy/
https://www.wittetools.com/en/general-terms-conditions-of-delivery-and-payment/legal-notice/
['https://www.wittetools.com/en/gen

21
Query: site:"https://comipolaris.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fcomipolaris.com+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 6

22
Query: site:"https://www.comercole.it "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.comercole.it+%22privacy+policy&num=3
len alldata: 2
https://www.comercole.it/privacy-policy-ch.html
https://www.comercole.it/content/files/GDPR%20POLITICA-PRIVACY%20ingl.pdf
['https://www.comercole.it/privacy-policy-ch.html', 'https://www.comercole.it/content/files/GDPR%20POLITICA-PRIVACY%20ingl.pdf']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.comercole.it/privacy-policy-ch.html
Found relevant terms in URL! Succesful break!
URL count: 7

23
Query: site:"https://www.bahco.com/int_en "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.bahco.com%2Fint

42
Query: site:"http://www.homberger.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.homberger.com+%22privacy+policy&num=3
len alldata: 2
https://www.homberger.com/en/qualita/
https://www.homberger.com/en/news/e-on-line-il-nuovo-sito-web/
['https://www.homberger.com/en/qualita/', 'https://www.homberger.com/en/news/e-on-line-il-nuovo-sito-web/']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.homberger.com/en/qualita/
Assessing privacy policy URL: https://www.homberger.com/en/news/e-on-line-il-nuovo-sito-web/
No results. Breaking ..
URL count: 15

43
Query: site:"https://www.sam-outillage.fr "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.sam-outillage.fr+%22privacy+policy&num=3
len alldata: 2
https://www.sam-outillage.fr/mentions-legales.html
https://www.sam-outillage.fr/sam-outillage-kapsam-privacy.htm
['https://www.sam-outillage.fr/mentions-legales.html', 'htt

len alldata: 2
https://www.drapertools.com/privacy-policy
https://www.drapertools.com/cookies-policy
['https://www.drapertools.com/privacy-policy', 'https://www.drapertools.com/cookies-policy']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.drapertools.com/privacy-policy
Found relevant terms in URL! Succesful break!
URL count: 23

65
Query: site:"http://www.gothamstaple.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.gothamstaple.com+%22privacy+policy&num=3
len alldata: 2
[]
Considering 0 URL(s) ...
URL count: 23

66
Query: site:"http://www.gratomat.de "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.gratomat.de+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 23

67
Query: site:"https://remingtonpowertools.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fremingtonpowertools.com+%22pr

86
Query: site:"https://www.grupp.de "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.grupp.de+%22privacy+policy&num=3
len alldata: 1
https://www.grupp.de/datenschutz
['https://www.grupp.de/datenschutz']
Considering 1 URL(s) ...
Assessing privacy policy URL: https://www.grupp.de/datenschutz
No results. Breaking ..
URL count: 30

87
Query: site:"https://www.jrboone.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.jrboone.com+%22privacy+policy&num=3
len alldata: 2
https://www.jrboone.com/privacy-policy
https://www.jrboone.com/industry/powders-oils
['https://www.jrboone.com/privacy-policy', 'https://www.jrboone.com/industry/powders-oils']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.jrboone.com/privacy-policy
Found relevant terms in URL! Succesful break!
URL count: 31

88
Query: site:"https://www.bkpowersystems.com "privacy policy
** Search query in URL: https://

...

len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 254

895
Query: site:"https://www.rcamerica.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.rcamerica.com+%22privacy+policy&num=3
len alldata: 2
https://www.rcamerica.com/cs-privacy-statement
https://www.rcamerica.com/cs_terms-of-use
['https://www.rcamerica.com/cs-privacy-statement', 'https://www.rcamerica.com/cs_terms-of-use']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.rcamerica.com/cs-privacy-statement
Found relevant terms in URL! Succesful break!
URL count: 255

896
Query: site:"http://www.tech-trade.ch "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.tech-trade.ch+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 255

897
Query: site:"http://www.zylin.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.zylin.com+%22privacy+

len alldata: 2
https://www.tacwise.com/privacy-policy/
https://www.tacwise.com/kd733/wp-content/uploads/2018/05/Job-Applicants-Privacy-Policy..pdf
['https://www.tacwise.com/privacy-policy/', 'https://www.tacwise.com/kd733/wp-content/uploads/2018/05/Job-Applicants-Privacy-Policy..pdf']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.tacwise.com/privacy-policy/
Found relevant terms in URL! Succesful break!
URL count: 262

917
Query: site:"https://catt-llc.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fcatt-llc.com+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 262

918
Query: site:"http://www.gbicincinnati.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.gbicincinnati.com+%22privacy+policy&num=3
len alldata: 1
[]
Considering 0 URL(s) ...
URL count: 262

919
Query: site:"https://www.brgmachinery.com "privacy policy
** Search query in 

937
Query: site:"http://www.decisiondetective.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.decisiondetective.com+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 266

938
URL count: 266

939
Query: site:"http://bugsplat.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fbugsplat.com+%22privacy+policy&num=3
len alldata: 2
https://docs.bugsplat.com/introduction/production/security-privacy-and-compliance/privacy-policy
https://docs.bugsplat.com/introduction/production/security-privacy-and-compliance
['https://docs.bugsplat.com/introduction/production/security-privacy-and-compliance/privacy-policy', 'https://docs.bugsplat.com/introduction/production/security-privacy-and-compliance']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://docs.bugsplat.com/introduction/production/security-privacy-and-compliance/privacy-policy
Found relevant terms in U

len alldata: 2
https://www.hicx.com/resources/
https://www.hicx.com/contact-us/
['https://www.hicx.com/resources/', 'https://www.hicx.com/contact-us/']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.hicx.com/resources/
Assessing privacy policy URL: https://www.hicx.com/contact-us/
No results. Breaking ..
URL count: 272

960
Query: site:"http://www.censornet.com "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.censornet.com+%22privacy+policy&num=3
len alldata: 2
https://www.censornet.com/privacy-policy/
https://www.censornet.com/wp-content/uploads/2021/01/Privacy-Policy-v1.3-December-2020.pdf
['https://www.censornet.com/privacy-policy/', 'https://www.censornet.com/wp-content/uploads/2021/01/Privacy-Policy-v1.3-December-2020.pdf']
Considering 2 URL(s) ...
Assessing privacy policy URL: https://www.censornet.com/privacy-policy/
Found relevant terms in URL! Succesful break!
URL count: 273

961
Query: site:"http://www.open

977
Query: site:"https://www.cmsgroup.net/home  "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.cmsgroup.net%2Fhome++%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 283

978
Query: site:"http://www.schubertsoftware.de "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22http%3A%2F%2Fwww.schubertsoftware.de+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 283

979
Query: site:"https://www.turbo-tec.eu "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fwww.turbo-tec.eu+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 283

980
Query: site:"https://kvtek.in "privacy policy
** Search query in URL: https://www.google.com/search?q=site%3A%22https%3A%2F%2Fkvtek.in+%22privacy+policy&num=3
len alldata: 0
[]
Considering 0 URL(s) ...
URL count: 283

981
Query: site:"https://www.muvenum.c

In [33]:
len(privacy_policies_url_list)

2468

In [35]:
(len([(collected_url) for collected_url in privacy_policies_url_list if collected_url is not ""]))

<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-35-be691173171a>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  (len([(collected_url) for collected_url in privacy_policies_url_list if collected_url is not ""]))


874

In [36]:
crunch_data['PP URL'] = privacy_policies_url_list

In [41]:
#save data
crunch_data.to_csv("crunch_data_collaborate_surv.csv", sep='\t', header=True, index=False)

In [42]:
crunch_data_r = pd.read_csv("crunch_data_collaborate_surv.csv", sep='\t', encoding='utf-8')

In [43]:
crunch_data_r

,Organization Name,Organization Name URL,Founded Date,Founded Date Precision,Number of Employees,Full Description,Website,Industries,Headquarters Location,Description,...,SEMrush - Visit Duration Growth,SEMrush - Visit Duration,SEMrush - Page Views / Visit,SEMrush - Page Views / Visit Growth,SEMrush - Bounce Rate,SEMrush - Bounce Rate Growth,SEMrush - Global Traffic Rank,SEMrush - Monthly Rank Change (#),SEMrush - Monthly Rank Growth,PP URL
0,Desire Group International limited,https://www.crunchbase.com/organization/desire...,2022-09-08,day,1-10,Desire Group International is a U.K based comp...,https://www.desiregroupinternational.co.uk/,"Business Development, Collaboration, Enterpris...","London, England, United Kingdom","Saas, Business development, Techsales, Marketi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.desiregroupinternational.co.uk/pri...
1,WorkHub Platform Inc.,https://www.crunchbase.com/organization/workhu...,2022-08-01,month,51-100,WorkHub is a tech company established in July ...,https://www.workhub.ai/,"B2B, Collaboration, Software","San Jose, California, United States",WorkHub is providing affordable team productiv...,...,14.91%,925,3.51,-13.63%,27.45%,-12.08%,"459,552","-1,880,548",-80.36%,https://www.workhub.ai/privacy-policy/
2,Neu Ocean Technologies,https://www.crunchbase.com/organization/neu-oc...,2022-02-24,day,101-250,Neu Ocean offers a comprehensive business mana...,https://neuocean.com,"Cloud Computing, Collaboration, Enterprise Sof...","London, England, United Kingdom",Scalable AI-powered business management and au...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Coleridge Initiative,https://www.crunchbase.com/organization/coleri...,2022-01-01,year,11-50,The Administrative Data Research Facility (ADR...,https://coleridgeinitiative.org/,"Analytics, Collaboration, Information Technology","Brooklyn, New York, United States",Coleridge Initiative is a company that provide...,...,47.32%,165,3.81,26.97%,6.36%,-80.92%,"6,062,300","2,483,928",69.42%,https://textbook.coleridgeinitiative.org/chap-...
4,Calliper,https://www.crunchbase.com/organization/calliper,2022-01-01,year,1-10,NaN,https://www.getcalliper.com/,"Analytics, Business Intelligence, Collaboratio...","London, England, United Kingdom",Making data accessible and actionable for ever...,...,NaN,191,2.92,NaN,10%,NaN,"3,743,703",NaN,NaN,https://www.getcalliper.com/privacy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,Woobius,https://www.crunchbase.com/organization/woobius,2007-10-02,day,1-10,Woobius is a collaboration tool for architects...,http://www.woobius.com,"Architecture, Collaboration, Construction, Ent...","London, England, United Kingdom",Woobius is a revolutionary collaboration hub f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.woobius.com/privacy-policy/
2464,Onehub,https://www.crunchbase.com/organization/onehub,2007-10-15,day,11-50,Onehub's mission is to provide small and mediu...,https://www.onehub.com/home,"Collaboration, Document Management, Enterprise...","Seattle, Washington, United States","Securely store, organize, and share files in t...",...,169.62%,"2,219",2.09,-16.63%,73.4%,33.36%,"80,534","-18,092",-18.34%,NaN
2465,eTask.it,https://www.crunchbase.com/organization/etask-...,2007-11-01,day,11-50,eTask.it harnesses the global trend of Web 2.0...,http://www.etask.it,"Collaboration, IT Management, Outsourcing, Pro...","Farnborough, Hampshire, United Kingdom",eTask is a U.K.-based project management solut...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,Combionic,https://www.crunchbase.com/organization/combionic,2007-11-01,day,11-50,Combionic collaboration software connects peop...,http://www.combionic.com,"Collaboration, Content, Risk Management, Software","Berlin, Berlin, Germany",Combionic develops gateway technology to conne...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 3: Scrape privacy policies

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
def scrape_policies_google(url):
    policies = []
    sentences = []    
    try:
        
        article = Article(url)
#             print(url)
        article.download() #Downloads the link’s HTML content
#             print(url)
        article.parse() #Parse the article
#             print(url)
#                 print(article.title)
        doc = nlp(article.text)
        print("PP language = EN?: " + str(detect(article.text) == 'en'))
        print("PP length > 10 sentences?: " + str(len(list(doc.sents)) > 10))

        if detect(article.text) == 'en' and len(list(doc.sents)) > 10:
            print("Policy meets requirements of language and length ... ")
            sentences = list(doc.sents)
            print("Scraping successful!")

        else:
            print("Scraping not successful")
    except:
            pass
    print()
    return sentences

In [ ]:
pp_list_sentences = []
for i, pp_url in enumerate(privacy_policies_url_list):
    print(i)
    if pp_url == "":
        pp_list_sentences.append("")
    else:
        pp_list_sentences.append(scrape_policies_google(pp_url))

In [ ]:
[print(len(pp)) for pp in pp_list_sentences]

# Step 4: Classification

In [ ]:
crunch_data_r

In [ ]:
GDPR_classes = ['DPO', 'Purpose', 'Acquired data', 'Data sharing', 'Rights']

In [ ]:
thresholds = [0.014130434782608696, 0.035326086956521736, 0.017934782608695653, 0.03369565217391304, 0.009782608695652175]

#### Preprocessing

In [ ]:
def preprocessing(pps):
#     tokenizer = nlp.tokenizer
    # tokenize sentences
    tokenized_sent = [sent.text.split() for sent in pps]
    
    # remove punctuation
    tokenized_sent = [[re.sub('[,’\'\.!?&“”():*_;"]', '', y) for y in x] for x in tokenized_sent]
    
    # remove words with numbers in them
    tokenized_sent = [[y for y in x if not any(c.isdigit() for c in y)] for x in tokenized_sent]
    
    # remove stopwords   
    tokenized_sent_clean = tokenized_sent
#     tokenized_sent_clean = [[y for y in x if y not in stopwords.words('english')] for x in tokenized_sent]
    
    # from nltk.stem import PorterStemmer
    porter = PorterStemmer()
    tokenized_sent_clean = [[porter.stem(y) for y in x] for x in tokenized_sent_clean]
    
#     lemmatizer = WordNetLemmatizer()
#     tokenized_sent_clean = [[lemmatizer.lemmatize(y) for y in x] for x in tokenized_sent_clean]

    
    detokenized_pps = []
    for i in range(len(tokenized_sent_clean)):
        t = ' '.join(tokenized_sent_clean[i])
        detokenized_pps.append(t) 
    
    return detokenized_pps

In [ ]:
def set_GDPR_columns(df):
    df['DPO'] = 0
    df['Purpose'] = 0
    df['Acquired data'] = 0
    df['Data sharing']  = 0
    df['Rights'] = 0

In [ ]:
set_GDPR_columns(crunch_data_r)

In [ ]:
pp_list_sentences_prep = []

for j, pp in enumerate(pp_list_sentences):
    pp_list_sentences_prep.append(preprocessing(pp))

In [ ]:
pp_list_sentences_prep

In [ ]:
crunch_data_r['PP text'] = pp_list_sentences_prep

In [ ]:
crunch_data_r

In [ ]:
crunch_data_r.to_csv("crunch_data_pp_url_text.csv", sep='\t', header=True)

#### Classification

In [ ]:
crunch_data_r = pd.read_csv("crunch_data_pp_url_text.csv", sep='\t', encoding='utf-8', index_col = 0)

In [ ]:
crunch_data_r

In [ ]:
crunch_data_r_selected = crunch_data_r.loc[crunch_data_r['PP text'] != "[]"]

In [ ]:
crunch_data_r_selected

In [ ]:
for index, row in crunch_data_r_selected.iterrows(): 
    x = row["PP text"]
    pp_text_split = x.split(', ')
    
    for j, category in enumerate(GDPR_classes):
             # Load from file to check if everything is ok
        filen = "linreg-oversampling-" + category + ".pkl"      
        with open(filen, 'rb') as file:
            vectorizer, lr = pickle.load(file)
            x = vectorizer.transform(pp_text_split)
        
            y_pred = lr.predict(x)
#             print(y_pred)
            n_pos_pred = list(y_pred).count(1)
#             print(n_pos_pred)
            
            
#             print("(" + str(n_pos_pred) + "/" + str(len(pp_text_split)) + ") >= " + str(thresholds[j]))
            if (n_pos_pred/len(pp_text_split)) >= thresholds[j]:
    #           MARK THE LABEL AS POSITIVE (1), DEFAULT STATE IS NEGATIVE (0)
#                 print("TRUE")
                crunch_data_r_selected.at[index, GDPR_classes[j]] = 1
            else:
                pass

In [ ]:
crunch_data_r_selected

# Classification Analysis (425 privacy policies)

In [ ]:
for idx, GDPR_class in enumerate(GDPR_classes):
    print(GDPR_class)
    print("Positively classified:" + str(crunch_data_r_selected[GDPR_class].value_counts()[0]) + " (" + str((crunch_data_r_selected[GDPR_class].value_counts()[0]/crunch_data_r_selected.shape[0])*100) + "%)")
    print("Negatively classified:" + str(crunch_data_r_selected[GDPR_class].value_counts()[1]) + " (" + str((crunch_data_r_selected[GDPR_class].value_counts()[1]/crunch_data_r_selected.shape[0])*100) + "%)")
    classification_analysis = [
       [GDPR_class, crunch_data_r_selected.shape[0], crunch_data_r_selected[GDPR_class].value_counts()[0], crunch_data_r_selected[GDPR_class].value_counts()[1]],
#        [GDPR_labels[idx], 'L1', 'numerical', 'full data', sm_lr_numpredictors_acc[idx], {k:v for (k,v) in dict(sm_lr_numpredictors[idx].pvalues).items() if ((v <= 0.05) and ( v != 0) and (k != 'const'))}]
      ]
    classification_analysis = pd.DataFrame(classification_analysis, columns =['GDPR Class', '# companies', 'Postive', 'Negative'])
#     print(summary_sm_sk.to_markdown())
    
    display(HTML(classification_analysis.to_html(index=False)))
    print()
    print()

# Statistical Analysis

### Select potentially interesting predictors

- Employee (object), 
- Type (object), 
- Founded Date (object), 
- Location
- Operating Status (object), 
- Industry 1 (object)

In [ ]:
pd_stats = crunch_data_r_selected[["Employees", "Founded Date", "Location", "Industry 1", "DPO", "Purpose", "Acquired data", "Data sharing", "Rights"]].copy(deep=True)

In [ ]:
pd_stats.info()

##### Employees

In [ ]:
pd_stats["Employees"].value_counts()

##### Founded Date

In [ ]:
pd_stats["Founded Date"].value_counts()

Convert to year

In [ ]:
f_date = pd_stats["Founded Date"].tolist()

In [ ]:
f_date_clean = [re.findall(r'(\d{4})', date)[0] if date is not np.nan else (np.nan) for date in f_date]

In [ ]:
len(f_date)

In [ ]:
(f_date_clean)

In [ ]:
pd_stats["Founded Year"] = f_date_clean

In [ ]:
pd_stats["Founded Year"].value_counts()

##### Location          

In [ ]:
pd_stats["Location"].value_counts()

In [ ]:
location = pd_stats["Location"].to_list()

In [ ]:
country = [(country.split(", ")[-1]) for country in location]

In [ ]:
len(country)

In [ ]:
pd_stats["Country"] = country

In [ ]:
pd_stats["Country"].value_counts()

##### Industry 1

In [ ]:
pd_stats["Industry 1"].value_counts()

### Drop old columns

In [ ]:
pd_stats.drop(['Founded Date', 'Location'], axis=1, inplace=True)

In [ ]:
pd_stats.info()

### Cast to category

In [ ]:
# Define the lambda function: categorize_label
label_categorical = lambda x: x.astype('category')

In [ ]:
pd_stats = pd_stats.apply(label_categorical, axis=0)

In [ ]:
pd_stats.dtypes

### LR with Statsmodels

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

# scaling
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

In [ ]:
pd_stats.isna().sum()

In [ ]:
GDPR_classes

#### Explore correlations

In [ ]:
X = pd_stats.drop(GDPR_classes,axis=1) # independant features
X = pd.get_dummies(X, drop_first = True)
sns.clustermap(X.corr())

#### Split data

In [ ]:
train, test = train_test_split(pd_stats, test_size=0.2, random_state=42)
X_train = train.drop(GDPR_classes,axis=1) # independant features

#### Encode non-numerical categorical data, and drop first to avoid collinearity

In [ ]:
X_train = pd.get_dummies(X_train, drop_first = True)

# Parameter Optimization

#### First without PO

In [ ]:
train, test = train_test_split(pd_stats, test_size=0.25, random_state=25)
sel_alpha_list = dict()
acc_last = 0

In [ ]:
y_train = train[GDPR_classes[0]] # dependant variable
y_test = test[GDPR_classes[0]] # dependant variable

In [ ]:
# independent features
X_train = train.drop(GDPR_classes, axis=1) 
# encode non-numerical categorical data, and drop first to avoid collinearity
X_train = pd.get_dummies(X_train, drop_first = True)

X_test = test.drop(GDPR_classes, axis=1) # independant features
X_test = pd.get_dummies(X_test, drop_first = True)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [ ]:
X_train

In [ ]:
model = sm.Logit(y_train,X_train)
logit_model = model.fit()

In [ ]:
pred_train = logit_model.predict(X_train)>=.5
pred_test = logit_model.predict(X_test)>=.5

In [ ]:
acc_train = (y_train==pred_train).mean()
acc_test = (y_test==pred_test).mean()

print("Acc: ", acc_test)
print("Alpha: ", alpha_op)

In [ ]:
alpha_list = list(np.arange(0.001, 10, 0.1))

##### Optimize parameters

In [ ]:
opt_alpha = optimize_logit(pd_stats, True, alpha_list, True)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
def optimize_logit(pd_stats, reg, alpha_range, intercept_set):
    train, test = train_test_split(pd_stats, test_size=0.2, random_state=25)
    sel_alpha_list = dict()
    acc_last = 0

    for GDPR_cat in GDPR_classes:
        alpha_sel = alpha_range[0]
        acc_last = 0

        print("***************** NEW ROUND!")
        for alpha_op in alpha_range:
            print("GDPR-category: " + GDPR_class)

            y_train = train[GDPR_class] # dependant variable
            y_test = test[GDPR_class] # dependant variable
            
#             sys.exit(0)

            # independent features
            X_train = train.drop(GDPR_classes, axis=1) 
            # encode non-numerical categorical data, and drop first to avoid collinearity
            X_train = pd.get_dummies(X_train, drop_first = True)

            X_test = test.drop(GDPR_classes, axis=1) # independant features
            X_test = pd.get_dummies(X_test, drop_first = True)

            if(intercept_set):
                X_train = sm.add_constant(X_train)
                X_test = sm.add_constant(X_test)
                
            print(y_train)

            print("flag 1")
            model = sm.Logit(y_train,X_train)
            print("flag 2")

            if(reg):
                logit_model = model.fit_regularized(method = 'l1', trim_mode = 'size', alpha = alpha_op)
            else:
                logit_model = model.fit()

            print("flag 3")

            pred_train = logit_model.predict(X_train)>=.5

            pred_test = logit_model.predict(X_test)>=.5

            acc_train = (y_train==pred_train).mean()

            acc_test = (y_test==pred_test).mean()
            
            print("Acc: ", acc_test)
            print("Alpha: ", alpha_op)

            sys.exit(0)
            if(acc_test >= acc_last):
                print("Alpha selected!")
                alpha_sel = alpha_op 
                acc_last = acc_test

            # last alpha in range? Place optimized alpha and accuracy in dict
            if(alpha_op == alpha_list[-1]):
                sel_alpha_list[GDPR_class] = [alpha_sel, acc_last]
            
            print()
            print()

    return sel_alpha_list